# 모델 해석

Azure Machine Learning을 사용하면 각 기능이 예측된 레이블에 주는 영향의 정도를 수치로 표시하는 *설명자*를 사용해 모델을 해석할 수 있습니다. 각기 다른 종류의 모델링 알고리즘에 적합한 여러 가지 일반 설명자가 있는데, 이러한 설명자를 사용하는 기본적인 방식은 동일합니다.

## SDK 패키지 설치

이 Notebook의 코드를 실행하려면 최신 버전의 **azureml-sdk** 및 **azureml-widgets** 패키지 외에 **azureml-explain-model** 패키지도 필요합니다. 그리고 이 Notebook에서는 Azure ML Interpretability 라이브러리(**azureml-interpret**)도 사용합니다. 이 라이브러리를 사용하면 일반적인 여러 종류의 모델을 해석할 수 있습니다. Azure ML 실험에서 학습되지 않았거나 Azure ML 작업 영역에 등록되지 않은 모델도 해석 가능합니다.

아래 셀의 명령을 실행하여 이러한 패키지가 설치되어 있는지 확인합니다. 

In [ ]:
!pip show azureml-explain-model azureml-interpret

## 모델 설명

먼저 Azure Machine Learning 외부에서 학습된 모델의 설명을 생성해 보겠습니다. 아래 셀을 실행하여 의사 결정 트리 분류 모델의 학습을 진행합니다.

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 당뇨병 데이터 세트 로드
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# 기능 및 레이블 분리
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

학습 프로세스에서는 보류 유효성 검사 데이터 세트를 기반으로 하여 몇 가지 모델 평가 메트릭이 생성되며, 이 메트릭을 통해 모델의 예측 정확도를 파악할 수 있습니다. 이번에는 데이터의 기능이 예측에 영향을 주는 방식을 알아보겠습니다.

### 모델의 설명자 가져오기

앞에서 설치한 Azure ML Interpretability 라이브러리에서 모델에 적합한 설명자를 가져옵니다. 설명자에는 여러 가지 종류가 있습니다. 이 예제에서는 *테이블 형식 설명자*를 사용합니다. 이 설명자는 적절한 [SHAP](https://github.com/slundberg/shap) 모델 설명자를 호출해 여러 종류의 모델을 설명하는 데 사용할 수 있는 "블랙 박스" 설명자입니다.

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# "기능" 및 "클래스" 필드는 선택 사항입니다.
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### *글로벌* 특징 중요도 가져오기

먼저 전반적인 *특징 중요도*를 평가하여 모델을 설명해야 합니다. 즉, 전체 학습 데이터 세트를 기준으로 하여 각 특징이 예측에 영향을 주는 범위를 수치로 표시해야 합니다.

In [ ]:
# 여기서는 학습 데이터 또는 테스트 데이터를 사용할 수 있습니다.
global_tab_explanation = tab_explainer.explain_global(X_train)

# 중요도를 기준으로 주요 기능 가져오기
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

특징 중요도에는 순위가 지정되며 가장 중요한 특징이 목록에서 맨 위에 표시됩니다.

### *로컬* 특징 중요도 가져오기

전반적인 특징 중요도를 설명한 후에는 개별 관찰 내용을 설명해야 합니다. 이번에는 개별 예측의 *로컬* 설명을 생성해 보겠습니다. 이 설명에는 사용 가능한 각 레이블 값을 예측하는 결정에 각 기능이 영향을 준 범위가 수치로 표시됩니다. 여기서는 이진 모델이 사용되므로 사용 가능한 레이블은 두 가지(당뇨병/당뇨병 아님)입니다. 그리고 데이터 세트의 개별 관찰에서 각 기능이 이러한 각 레이블 값에 주는 영향을 수치로 표시할 수 있습니다. 여기서는 테스트 데이터 세트의 처음 두 사례만 평가합니다.

In [ ]:
# 설명하려는 관찰 내용(처음 2개 항목) 가져오기
X_explain = X_test[0:2]

# 예측 가져오기
predictions = model.predict(X_explain)

# 로컬 설명 가져오기
local_tab_explanation = tab_explainer.explain_local(X_explain)

# 사용 가능한 각 레이블의 기능 이름 및 중요도 가져오기
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

## 모델 학습 실험에 설명 가능성 추가

앞에서 살펴본 것처럼 Azure Machine Learning 외부에서 학습된 모델의 설명을 생성할 수 있습니다. 하지만 Azure Machine Learning 작업 영역에서 실험을 사용하여 모델 학습과 등록을 진행할 때는 모델 설명을 생성하여 기록할 수 있습니다.

다음 셀의 코드를 실행하여 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 실험을 사용하여 모델 학습 및 설명

이제 실험을 만들고 실험에 필요한 파일을 로컬 폴더에 저장해 보겠습니다. 여기서는 앞에서 사용했던 것과 같은 당뇨병 데이터가 포함된 CSV 파일을 사용해 모델을 학습시킬 것입니다.

In [ ]:
import os, shutil
from azureml.core import Experiment

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# 실험 폴더에 데이터 파일 복사
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

이번에는 다른 Azure ML 학습 스크립트와 비슷한 학습 스크립트를 작성하겠습니다. 단, 이 스크립트에는 다음 기능이 포함됩니다.

- 이전 연습에서 모델 설명을 생성하는 데 사용한 것과 같은 라이브러리를 가져와 글로벌 설명을 생성하는 데 사용합니다.
- **ExplanationClient** 라이브러리를 사용하여 실험 출력에 설명을 업로드합니다.

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Azure ML 실행 라이브러리 가져오기
from azureml.core.run import Run

# 모델 설명용 라이브러리 가져오기
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# 기능 및 레이블 분리
X, y = data[features].values, data['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# 설명 가져오기
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# 설명 클라이언트 가져오기 및 설명 업로드
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# 실행 완료
run.complete()

이제 실험을 실행할 수 있습니다. **azureml-interpret** 라이브러리가 학습 환경에 포함되어 있으므로 스크립트는 **TabularExplainer**를 작성할 수 있으며 **ExplainerClient** 클래스를 사용할 수 있습니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 실험용 Python 환경 만들기
explain_env = Environment("explain-env")

# 패키지 종속성 집합 만들기(azureml-interpret 패키지 포함)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
explain_env.python.conda_dependencies = packages

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env) 

# 실험 제출
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

## 특징 중요도 값 검색

실험 실행이 완료되면 **ExplanationClient** 클래스를 사용하여 실험용으로 등록된 설명에서 특징 중요도를 검색할 수 있습니다.

In [ ]:
from azureml.interpret import ExplanationClient

# 기능 설명 가져오기
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 전체 특징 중요도
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studio에서 모델 설명 확인

실행 세부 정보 위젯의 **실행 세부 정보 보기** 링크를 클릭하여 Azure Machine Learning Studio에서 실행을 확인하고 **설명** 탭을 표시할 수도 있습니다. 그런 후에 다음 단계를 수행합니다

1. 테이블 형식 설명자의 설명 ID를 선택합니다.
2. 전반적인 글로벌 특징 중요도가 표시되는 **집계 특징 중요도** 차트를 확인합니다.
3. 테스트 데이터의 각 데이터 요소가 표시되는 **개별 특징 중요도** 차트를 확인합니다.
4. 개별 요소를 선택하여 선택한 데이터 요소의 개별 예측 로컬 특징 중요도를 확인합니다.
5. **새 집단** 단추를 사용하여 데이터 하위 집합을 다음 설정으로 정의합니다.
    - **데이터 세트 집단 이름**: Under 25s
    - **필터 선택**: 데이터 세트
        - 25세 미만(새 집단을 저장하기 전에 이 필터를 추가해야 함)
6. Age에서 25보다 큰 숫자를 필터링하는 **25 and over**라는 두 번째 새 집단을 만듭니다.
6. **집계 특징 중요도** 시각화를 검토하고, 정의한 두 집단의 상대적 특징 중요도를 비교합니다. 집단을 비교할 수 있기 때문에 여러 데이터 인구 하위 집합에서 특징이 예측에 미치는 영향이 어떻게 달라지는지 볼 수 있습니다.



**자세한 정보**: Azure ML에서 설명자를 사용하는 방법에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability)를 참조하세요. 